In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from matplotlib import pyplot as plt 
from sklearn.model_selection import KFold
import tensorflow as tf
import cv2 as cv
import os 
import math
import numpy as np
from time import perf_counter

In [ ]:
os.listdir() 

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

In [ ]:
#img_height, img_width, img_channel = cv.imread(f'{os.path.abspath(os.getcwd())}/train/i_top_2/Yen_11.png').shape
#img_shape = (img_height, img_width, img_channel)

#tmp_height = math.floor(img_height/4)
#tmp_width = math.floor(img_width/4)
shape_hyperparam_dir = (224, 224)
shape_hyperparam_cnn = (224, 224, 3) # Channel is unchanged

In [ ]:
train = ImageDataGenerator(rescale = 1/255)

In [ ]:
train_dataset = train.flow_from_directory(f'{os.path.abspath(os.getcwd())}/100', # Augmented. Only for 100.
                                          target_size = shape_hyperparam_dir,
                                          batch_size = 2**3, # Higher batch size is better but slower
                                          class_mode = "categorical"
                                         )

In [ ]:
train_dataset.class_indices

In [ ]:

model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(250, (3, 3), activation = "relu", input_shape = shape_hyperparam_cnn), #ConvLayer
                                    tf.keras.layers.MaxPool2D(2, 2), 

                                    tf.keras.layers.Conv2D(300, (4, 4), activation = "relu"),
                                    tf.keras.layers.MaxPool2D(2, 2),

                                    tf.keras.layers.Conv2D(500, (4, 4), activation = "relu"),
                                    tf.keras.layers.MaxPool2D(2, 2), 

                                    tf.keras.layers.Conv2D(256, (2,2), activation ="relu", kernel_regularizer='l1_l2', bias_regularizer="l1", activity_regularizer="l2"),

                                    tf.keras.layers.Conv2D(800, (3, 3), activation = "relu"),    # This 
                                    tf.keras.layers.MaxPool2D(2, 2),                             # This 

                                    tf.keras.layers.Conv2D(256, (2, 2), activation = "relu"),
                                    tf.keras.layers.MaxPool2D(2, 2), 

                                    tf.keras.layers.Conv2D(512, (3, 3), activation = "relu"),    # This 
                                    tf.keras.layers.MaxPool2D(2, 2),                             # This  

                                    tf.keras.layers.BatchNormalization(),

                                    tf.keras.layers.Flatten(),

                                    tf.keras.layers.Dense(500, activation = "relu"),             # This 

                                    tf.keras.layers.Dense(250, activation = "relu"),

                                    tf.keras.layers.Dense(100, activation = "relu", kernel_regularizer='l1_l2', bias_regularizer="l1_l2", activity_regularizer="l1_l2"),

                                    tf.keras.layers.BatchNormalization(),

                                    # FINAL DENSE LAYER NEURON COUNT SHOULD BE EQUAL TO OUR CLASSESS (n = 8)
                                    tf.keras.layers.Dense(8, activation = "softmax"), 

])


In [ ]:
model.compile(loss = "categorical_crossentropy",
              optimizer = Adam(learning_rate = 0.0001),
              metrics=['accuracy']
)

In [ ]:

start = perf_counter()
history = model.fit(train_dataset,
          epochs = 150,
)
end = perf_counter()
print("----------------------TRAINING FINISHED------------")
print(f" Training Time = {math.floor((end-start)/3600) } hours and {math.floor((end-start)/60)%60} mins")


In [ ]:
print(f" Main Model Training Time = {math.floor((end-start)/3600) } hours and {math.floor((end-start)/60)%60} mins")

In [ ]:
path_to_model = f'{os.path.abspath(os.getcwd())}/main_model'

model.save(f'{path_to_model}/model.h5')

hist_file = open(f'{path_to_model}/model_latestaccu.txt', 'a')
hist_file.write(f"latest_accuracy: {history.history['accuracy'][-1]}")
hist_file.close()

hist_file2 = open(f'{path_to_model}/model_history.txt', 'a')
hist_file2.write(f"{history.history}")
hist_file2.close()

for metric in history.history:
  x_label = "Epoch"
  y_label = ""
  title = ""
  color = "green"

  if metric == "loss":
    title = "Training Loss"
    y_label = "Loss"
    color = "blue"

  elif metric ==  "accuracy":
    title = "Training Accuracy"
    y_label = "Accuracy"
    color = "red"

  else:
    title = metric
    y_label = metric


  plt.plot(history.history[metric], color = color)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.title(title)
  plt.savefig(f'{path_to_model}/{metric}.png')
  plt.show()